# 自动微分

深度学习框架通过自动计算导数，即**自动微分**（automatic differentiation）来加快求导。
实际中，根据设计好的模型，系统会构建一个**计算图**（computational graph），
来跟踪计算是哪些数据通过哪些操作组合起来产生输出。
自动微分使系统能够随后反向传播梯度。
这里，**反向传播**（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

## 一个简单的例子

**假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导**。
首先，我们创建变量`x`并为其分配一个初始值。

In [14]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

**在我们计算$y$关于$\mathbf{x}$的梯度之前，需要一个地方来存储梯度。**

重要的是，我们不会在每次对一个参数求导时都分配新的内存。
因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。
注意，一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。


In [15]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

现在计算$y$。


In [16]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

接下来，**通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度**，并打印这些梯度。

注：$\nabla_{\mathbf{x}} 2\mathbf{x}^{\top}\mathbf{x} = 4\mathbf{x}$

In [17]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

让我们快速验证这个梯度是否计算正确。

In [18]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算`x`的另一个函数。


In [19]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
g = x.sum()
g.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播

当`y`不是标量时，向量`y`关于向量`x`的导数的最自然解释是一个**矩阵**。
对于高阶和高维的`y`和`x`，求导的结果可以是一个**高阶张量**。

当调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
这里，**我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。**


```
torch.autograd.backward(
		tensors, 
		grad_tensors=None, 
		retain_graph=None, 
		create_graph=False, 
		grad_variables=None)
```

注：`torch.autograd.backward(z) == z.backward()`

- `tensor`: 用于计算梯度的tensor。
- `grad_tensors`: 可以简单地理解成在求梯度时的权重，一般是设为1。也是一个tensor，shape一般需要和前面的tensor保持一致。
- `retain_graph`: 通常在调用一次backward后，pytorch会自动把计算图销毁，所以要想对某个变量重复调用backward，则需要将该参数设置为True
- `create_graph`: 当设置为True的时候可以用来计算更高阶的梯度

注：`.backward()`只能对**标量输出**求导

```
z.backward(y) # y和z的shape相同，都是向量
torch.dot(z , y).backward() # 等价
torch.sum(z * y).backward() # 等价
```

In [21]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
y.sum().backward(retain_graph=True) # 等价于y.backward(torch.ones(len(x)))
print(x.grad)

x.grad.zero_()
y.backward(torch.ones(len(x)), retain_graph=True)
print(x.grad)

tensor([0., 2., 4., 6.])
tensor([0., 2., 4., 6.])


## 分离计算

有时，我们希望[**将某些计算移动到记录的计算图之外**]。
例如，假设`y`是作为`x`的函数计算的，而`z`则是作为`y`和`x`的函数计算的。
想象一下，我们想计算`z`关于`x`的梯度，但由于某种原因，希望将`y`视为一个常数，
并且只考虑到`x`在`y`被计算后发挥的作用。

这里可以分离`y`来返回一个新变量`u`，该变量与`y`具有相同的值，
但丢弃计算图中如何计算`y`的任何信息。
换句话说，梯度不会向后流经`u`到`x`。
因此，下面的反向传播函数计算`z=u*x`关于`x`的偏导数，同时将`u`作为常数处理，
而不是`z=x*x*x`关于`x`的偏导数。


`tensor.detach()`返回一个新的tensor，从当前计算图中分离下来的，但是仍指向原变量的存放位置,不同之处只是requires_grad为false，得到的这个tensor永远不需要计算其梯度，不具有grad。

使用detach返回的tensor和原始的tensor**共享同一个内存**，即一个修改另一个也会跟着改变。

In [24]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
print(x.grad)
x.grad == u

tensor([0., 1., 4., 9.])


tensor([True, True, True, True])

由于记录了`y`的计算结果，我们可以随后在`y`上调用反向传播，
得到`y=x*x`关于的`x`的导数，即`2*x`。


In [25]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Python控制流的梯度计算

使用自动微分的一个好处是：
**即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度**。

在下面的代码中，`while`循环的迭代次数和`if`语句的结果都取决于输入`a`的值。


In [26]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

让我们计算梯度。


In [28]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

我们现在可以分析上面定义的`f`函数。
对于任何`a`，存在某个常量标量`k`，使得`f(a)=k*a`，其中`k`的值取决于输入`a`，因此可以用`d/a`验证梯度是否正确。


In [29]:
a.grad == d / a

tensor(True)

## 小结

深度学习框架可以自动计算导数：我们首先将**梯度附加**到想要对其计算偏导数的**变量**上，然后**记录目标值的计算**，执行它的**反向传播**函数，并访问**得到的梯度**。

## 练习

1. 为什么计算二阶导数比一阶导数的开销要更大？
2. 在运行反向传播函数之后，立即再次运行它，看看会发生什么。
- `RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.`


3. 在控制流的例子中，我们计算`d`关于`a`的导数，如果将变量`a`更改为随机向量或矩阵，会发生什么？

In [30]:
a = torch.randn(size=(1,2), requires_grad=True)
d = f(a)
d.sum().backward()
print(a.grad)
print(a.grad == d / a)

tensor([[102400., 102400.]])
tensor([[True, True]])


4. 重新设计一个求控制流梯度的例子，运行并分析结果。

In [33]:
import math
def h(x):
    for i in range(3):
        x = torch.exp(x)
    return x

i = torch.randn(size=(), requires_grad=True)
j = h(i)
j.backward()
print(i.grad,j)

tensor(6.2460) tensor(5.9910, grad_fn=<ExpBackward0>)



5. 使$f(x)=\sin(x)$，绘制$f(x)$和$\frac{df(x)}{dx}$的图像，其中后者不使用$f'(x)=\cos(x)$。

[Discussions](https://discuss.d2l.ai/t/1759)
